# Phylogeny for EMP500 after randomly removing genes to simulate completeness

## KOfamscan annotation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy.stats import pmean
import matplotlib.gridspec as grid_spec
import upsetplot
import os, sys

In [2]:
plt.rcParams.update({
    'figure.autolayout': True,
    'savefig.bbox': 'tight',
    'svg.fonttype': 'none',
    'font.sans-serif': 'Arial',
    'font.size': 12,
    'boxplot.medianprops.linewidth': 2,
    'boxplot.flierprops.markeredgecolor': 'none',
    'boxplot.flierprops.markersize': 5})

Functions

In [3]:
def adjMatv2(edges_ogs, edges_genomes):
    # Get unique elements
    ogs, ogs_indices = np.unique(edges_ogs, return_inverse = True)
    genomes, genomes_indices = np.unique(edges_genomes, return_inverse = True)
    # Calculate bin counts for each combination of indices
    counts = np.bincount(ogs_indices * len(genomes) + genomes_indices, minlength = len(ogs) * len(genomes))
    # Reshape counts as adjacency matrix
    adj = counts.reshape(len(ogs), len(genomes))
    return adj, genomes, ogs

In [4]:
def prop_single_copy(genes, adj, trans = False):
    
    selected_genes = adj[genes]
    
    single_copy = np.sum(selected_genes == 1, axis = 1)
    all_copies = np.sum(selected_genes >= 1, axis = 1)
    
    ratios = single_copy / all_copies
    
    if trans:
        ratios = np.arcsin(np.sqrt(ratios))
    
    return ratios

In [5]:
def save_genes(genes, fOut):
    with open(fOut, 'w') as f:
        for gene in genes:
            f.write(f'{gene}\n')

In [6]:
def save_markers_per_genome(markers_per_genome, fOut):
    with open(fOut, 'w') as f:
        for genome, number in markers_per_genome.items():
            f.write(f'{genome}:{number}\n')

In [7]:
def parse_kofamscan(file):
    '''
    Parse KOfamscan output file
    '''
    results = []
    with open(file, 'r') as f:
        for line in f:
            if not line.startswith('#'):
                tmp = re.sub('\s+', ' ', line.strip()).split(' ')
                # Omit the description column, ko_score, and e-value
                orf, ko, bit_score = tmp[0], tmp[1], float(tmp[3])
                results.append([orf, ko, bit_score])
    return results

In [8]:
def remove_genes(adj_mod):
    '''
    Function that removes genes if:
        present in less than 4 genomes/species
    '''
    remove = np.array([i for i in range(len(adj_mod)) if adj_mod[i].sum() < 4])
    return remove

In [9]:
def greedy_power_mean_sample_final(data, k, p, pseudocount, min_universality_genes):
    """Select k rows from a matrix such that the selection criterion by column is maximized.

    Parameters
    ----------
    data : ndarray (2D)
        Input data matrix.
    k : int
        Number of rows to select.
    p : float
        Exponent.

    Returns
    -------
    ndarray (1D)
        Row indices selected in order.
    """

    n, m = data.shape

    # Matrix is empty
    if n == 0 or m == 0:
        raise ValueError(f'Matrix is empty!')

    # Matrix contains only zeroes
    if (data == 0).all():
        raise ValueError(f'Matrix only contains 0\'s')

    if (data.sum(axis = 1) == min_universality_genes).all():
        raise ValueError(f'Matrix only contains genes present in less than {min_universality_genes}')

    if k >= n:
        raise ValueError(f'k should be smaller than {n}')
    
    # Add pseudocount
    data = data + pseudocount

    # cumulative gene counts
    counts = np.zeros(m, dtype = int)

    # gene indices in original data matrix
    indices = np.arange(n)

    # indices of selected genes
    selected = []

    # will select k genes sequentially
    for i in range(k):
        # calculate counts after adding each gene
        sums_ = counts + data

        # select a gene that maximizes the power mean gene count per genome, using the cumulative matric
        if isinstance(p, int) or isinstance(p, np.int64): 
            choice = pmean(sums_, int(p), axis = 1).argmax()
        elif p == 'min':
            choice = sums_.min(axis = 1).argmax()
        elif p == 'max':
            choice = sums_.max(axis = 1).argmax()
        else:
            raise ValueError(f'Invalid p: {p}.')

        # append index of selected gene
        selected.append(indices[choice])

        # update per-species gene counts
        counts = sums_[choice]

        # remove selected gene from data matrix
        data = np.delete(data, choice, axis = 0)

        # remove selected gene from indices
        indices = np.delete(indices, choice)

    return np.sort(np.array(selected))

Load bins statistics

In [10]:
df_bins = pd.read_csv(f'../markers_selection/input_data/emp/mags_statistics_kegg.tsv', index_col = 0, sep = '\t')
df_bins.shape

(811, 3)

**Remove bins outside the diagonal (`num_prokka_genes` vs `num_pfam_families`) and with less than 20 PFAM families**

In [11]:
to_exclude = ['SaltMarsh.Metabat.Bin.73', 'SaltMarsh.Metabat.Bin.70', 'SaltMarsh.Metabat.Bin.9', 'SaltMarsh.Metabat.Bin.33',
              'Steppe.Metabat.Bin.3', 'Mayer_watershed.Metabat.Bin.5', 'Jensen_Benthic.Metabat.Bin.3', 'Jensen_Benthic.Metabat.Bin.5',
              'Lichen.Metabat.Bin.11', 'Lichen.Metabat.Bin.19', 'Lichen.Metabat.Bin.29', 'Seedorf_soil.Metabat.Bin.1',
              'Seedorf_soil.Metabat.Bin.2', 'Seedorf_soil.Metabat.Bin.6', 'Marsh.Metabat.Bin.10',
              'Marsh.Metabat.Bin.12', 'Marsh.Metabat.Bin.35', 'Marsh.Metabat.Bin.7']
bins = df_bins[~df_bins.index.isin(to_exclude)].index.values

In [12]:
len(bins)

793

Load KOfamScan annotations

In [13]:
%%time
results = []
for bin in bins:
    # Load file
    with open(f'../markers_selection/input_data/emp/kegg/{bin}.tsv', 'r') as f:
        for line in f:
            row = line.strip().split('\t')
            orf, ko, bit_score = row[0], row[1], float(row[3])
            results.append([orf, ko, bit_score, bin])

CPU times: user 1.29 s, sys: 157 ms, total: 1.45 s
Wall time: 1.68 s


In [14]:
len(results)

839302

In [15]:
df = pd.DataFrame(results, columns = ['orf', 'ko', 'bit_score', 'bin'])
df.set_index('orf', inplace = True)
df['genome'] = df.index.map(lambda x: x.split('_')[0])

In [16]:
df.shape

(839302, 4)

In [17]:
orfs_all = set(df.index.values)
len(orfs_all)

803458

### Load genes to remove (incompleteness)

In [18]:
def remove_genes_completeness(bins, orfs_all, threshold, replicate):
    '''
    Add genes to bin.
    Parameters
    ----------
        bins : ndarray 1D
            Bin names
        orfs_all : set
            ORFs of all bins
        threshold : float
            Completeness 
        replicate : int
            Replicate number
    Returns
    -------
        to_keep : list
            ORFs to keep
    '''
    to_remove = []
    
    # Iterate over bins
    for bi in bins:
        # Add genes that will be removed remove
        sampled = np.loadtxt(f'./output_data/incompleteness/sampled_genes/{bi}_sample_contigs_genes_th_{threshold}_rep_{replicate}.txt', dtype = 'str')
        to_remove.extend(sampled)
        
    # Take difference
    to_keep = list(orfs_all.difference(set(to_remove)))

    return to_keep

# Using different completeness and copy number thresholds

In [19]:
bins.shape

(793,)

In [20]:
len(orfs_all)

803458

In [21]:
thresholds_completeness = [0.05, 0.1, 0.2, 0.4]
replicates = np.arange(10)
thresholds_copy_number = [1.0]
ps = [0]
k = 400

In [ ]:
%%time
for th_comp in thresholds_completeness:
    print(f'Completeness threshold: {th_comp}')
    for rep in replicates:
        print(f'\tReplicate: {rep}')
        # Load PFAM scan and filter based on completeness
        to_keep = remove_genes_completeness(bins, orfs_all, th_comp, rep)
        tmp_df = df.loc[to_keep]
        for th_copy in thresholds_copy_number:
            # print(f'\tCopy number threshold: {th_copy}')
            # Create directory
            path = f'./output_data/incompleteness/marker_genes/comp_th_{th_comp}/replicate_{rep}/copy_th_{th_copy}'
            os.system(f'mkdir -p {path}/orfs')
            os.system(f'mkdir -p {path}/proteins')
            os.system(f'mkdir -p {path}/presence_absence_copies')
            # Filter based on copy number, where the score is greater than or equal to max_score * threshold
            max_scores = tmp_df.groupby(['genome', 'ko'])['bit_score'].transform('max')
            filtered_df = tmp_df[tmp_df['bit_score'] >= max_scores * th_copy]
            # Build presence-absence matrix
            edges_genomes = filtered_df['genome'].values
            edges_ogs = filtered_df['ko'].values
            adj, genomes, ogs = adjMatv2(edges_ogs, edges_genomes)
            adj1 = adj
            adj2 = adj1.copy()
            adj1[adj1 > 1] = 1
            # Copy presence/absence matrix
            adj_mod = adj1.copy()
            # Select genes to remove
            remove = remove_genes(adj_mod)
            # Remove genes
            adj_mod = np.delete(adj_mod, remove, axis = 0)
            # Remove genes from list
            ogs_mod = np.delete(ogs, remove, axis = 0)
            # Copy matrix
            adj_mod2 = adj2.copy()
            # Remove genes
            adj_mod2 = np.delete(adj_mod2, remove, axis = 0)
            # Iterate over values of hyperparameter p
            all_genes = []
            for p in ps:
                # Select marker genes
                select_presence_absence_copies = greedy_power_mean_sample_final(adj_mod2, k, p = p, pseudocount = 0.1, min_universality_genes = 1)
                marker_genes = [ogs_mod[i] for i in select_presence_absence_copies]
                num_marker_genes_per_genome = {genome : number for genome, number in zip(genomes, adj_mod[select_presence_absence_copies].sum(axis = 0))}
                all_genes.extend(marker_genes)
                # Save 
                save_genes(marker_genes, f'{path}/presence_absence_copies/select_k_{k}_p_{p}.txt')
                save_markers_per_genome(num_marker_genes_per_genome, f'{path}/presence_absence_copies/num_markers_per_genome_k_{k}_p_{p}.txt')
                
            # Save union
            all_genes_union = set(all_genes)
            save_genes(all_genes_union, f'{path}/all_genes.txt') 
            # Save ORFs
            for gene in all_genes_union:
                orfs = filtered_df.query('ko == @gene').index.values
                with open(f'{path}/orfs/{gene}.txt', 'w') as f:
                    for orf in orfs:
                        f.write(f'{orf}\n')

Completeness threshold: 0.05
	Replicate: 0
	Replicate: 1
